In [1]:
import pandas as pd
import datetime
import re
from time import mktime
import numpy as np
data_file = pd.read_csv('main_task.xls')
cui = pd.read_csv('main_task.xls')
data_test = pd.read_csv('kaggle_task.csv')

In [2]:
data_file['Price Range'].value_counts()

$$ - $$$    18412
$            6279
$$$$         1423
Name: Price Range, dtype: int64

In [3]:
len(data_file['City'].value_counts())

31

In [4]:
cui['Cuisine Style'].fillna('Other', inplace = True)
cui['Cuisine Style'] = cui['Cuisine Style'].str.strip('[')
cui['Cuisine Style'] = cui['Cuisine Style'].str.strip(']')
cui['Cuisine Style'] = cui['Cuisine Style'].str.replace('\'','')
cui['Cuisine Style'] = cui['Cuisine Style'].str.split(',')

In [5]:
data_file['Cuisine Style'] = cui['Cuisine Style']
data_file = data_file.explode('Cuisine Style')
data_file['Cuisine Style'].str.strip(' ')
data_file = pd.get_dummies(data_file, columns=['Cuisine Style'], prefix='', prefix_sep='')

In [6]:
cui

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"[European, French, International]",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,[Other],1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"[Japanese, Sushi, Asian, Grill, Vegetarian...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,[Other],3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"[German, Central European, Vegetarian Friendly]",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"[Italian, Vegetarian Friendly, Vegan Options...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414
39996,id_6340,Paris,"[French, American, Bar, European, Vegetari...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036
39997,id_1649,Stockholm,"[Japanese, Sushi]",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615
39998,id_640,Warsaw,"[Polish, European, Eastern European, Centra...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838


City

In [7]:
data_file = pd.get_dummies(data_file, columns=['City'], prefix='', prefix_sep='')

In [8]:
pattern = re.compile("\d+\/\d+\/\d+")
df = data_file.copy()
df['Reviews'] = df['Reviews'].fillna('None')
df['date_rev'] = df['Reviews'].apply(pattern.findall)
df['date_rev'] = df['date_rev'].apply(
    lambda x: [x[-2], x[-1]] if len(x) >= 3 else x)
df['rev_1'] = df['date_rev'].apply(lambda x: x[0] if len(x) >= 1 else None)
df['rev_2'] = df['date_rev'].apply(lambda x: x[1] if len(x) == 2 else None)
df['rev_1'] = pd.to_datetime(df['rev_1'])
df['rev_2'] = pd.to_datetime(df['rev_2'])
print('Самая свежая дата', df[['rev_1', 'rev_2']].max().max())
print('Самая ранняя дата', df[['rev_1', 'rev_2']].min().min())
max_days = abs(df['rev_1'] - df['rev_2']).max()
print('Количество дней', max_days)

Самая свежая дата 2018-02-26 00:00:00
Самая ранняя дата 2004-04-21 00:00:00
Количество дней 3207 days 00:00:00


Ценовая категория

In [9]:
def price(x):
    if x == '$$ - $$$':
        x = 2
    elif x =='$':
        x = 1
    else:
        x = 3
    return x
data_file['Price Range'] = data_file['Price Range'].apply(price)

Reviews

In [10]:
data_file['days'] = abs(df['rev_1'] - df['rev_2']).dt.days
data_file['days'].fillna(data_file['days'].mean(),inplace = True)
df['rev_1'].fillna(df['rev_1'].mean(),inplace = True)
df['rev_1'] = df['rev_1'].apply(lambda x: mktime(x.timetuple()))
df['rev_2'].fillna(df['rev_2'].mean(),inplace = True)
df['rev_2'] = df['rev_2'].apply(lambda x: mktime(x.timetuple()))
data_file['rev_1'] = df['rev_1']
data_file['rev_2'] = df['rev_2']

In [11]:
data_file.drop(['Restaurant_id','Reviews', 'URL_TA', 'ID_TA'],axis = 1,inplace = True)
data_file['Ranking'].fillna(data_file['Ranking'].mean(),inplace = True)
data_file['Rating'].fillna(data_file['Rating'].mean(),inplace = True)
data_file['Number of Reviews'].fillna(data_file['Number of Reviews'].mean(),inplace = True)
X = data_file.drop(['Rating'], axis = 1)  

In [12]:
RANDOM_SEED = 42

In [13]:
data_file

,Ranking,Rating,Price Range,Number of Reviews,Afghani,African,Albanian,American,Arabic,Argentinean,...,Paris,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich,days,rev_1,rev_2
0,5570.0,3.5,2,194.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,41.0,1.514668e+09,1.511125e+09
0,5570.0,3.5,2,194.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,41.0,1.514668e+09,1.511125e+09
0,5570.0,3.5,2,194.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,41.0,1.514668e+09,1.511125e+09
1,1537.0,4.0,3,10.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,382.0,1.499288e+09,1.466284e+09
2,353.0,4.5,3,688.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,1.515359e+09,1.515186e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39998,641.0,4.0,2,70.0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,23.0,1.499720e+09,1.497733e+09
39998,641.0,4.0,2,70.0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,23.0,1.499720e+09,1.497733e+09
39998,641.0,4.0,2,70.0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,23.0,1.499720e+09,1.497733e+09
39998,641.0,4.0,2,70.0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,23.0,1.499720e+09,1.497733e+09


In [14]:
y = data_file['Rating']  
data_file.drop(['Rating'], axis = 1, inplace = True)
    # Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  
      
    # Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
    # Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [15]:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
      
# Создаём модель  
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)
  
    # Обучаем модель на тестовом наборе данных  
model.fit(X_train, y_train)  
      
    # Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
    # Предсказанные значения записываем в переменную y_pred  
y_pred = model.predict(X_test) 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   41.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.6min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


In [16]:
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.08502407054337462


In [17]:
data_test

,Restaurant_id,City,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_0,Paris,"['Bar', 'Pub']",12963.0,$$ - $$$,4.0,"[[], []]",/Restaurant_Review-g187147-d10746918-Reviews-L...,d10746918
1,id_1,Helsinki,"['European', 'Scandinavian', 'Gluten Free Opti...",106.0,$$ - $$$,97.0,"[['Very good reviews!', 'Fine dining in Hakani...",/Restaurant_Review-g189934-d6674944-Reviews-Ra...,d6674944
2,id_2,Edinburgh,['Vegetarian Friendly'],810.0,$$ - $$$,28.0,"[['Better than the Links', 'Ivy Black'], ['12/...",/Restaurant_Review-g186525-d13129638-Reviews-B...,d13129638
3,id_3,London,"['Italian', 'Mediterranean', 'European', 'Vege...",1669.0,$$$$,202.0,"[['Most exquisite', 'Delicious and authentic']...",/Restaurant_Review-g186338-d680417-Reviews-Qui...,d680417
4,id_4,Bratislava,"['Italian', 'Mediterranean', 'European', 'Seaf...",37.0,$$$$,162.0,"[['Always the best in bratislava', 'Very good ...",/Restaurant_Review-g274924-d1112354-Reviews-Ma...,d1112354
...,...,...,...,...,...,...,...,...,...
9995,id_9995,London,"['Caribbean', 'Barbecue', 'Jamaican', 'Vegetar...",4367.0,$,28.0,[['By far best Chicken burgers and wings in L....,/Restaurant_Review-g186338-d11916827-Reviews-C...,d11916827
9996,id_9996,Lisbon,"['Indian', 'Asian', 'Nepali', 'Bangladeshi']",2612.0,$$ - $$$,2.0,"[['Ok indian food'], ['11/25/2017']]",/Restaurant_Review-g189158-d12701633-Reviews-C...,d12701633
9997,id_9997,Milan,NaN,5562.0,NaN,2.0,"[[], []]",/Restaurant_Review-g187849-d13198194-Reviews-Q...,d13198194
9998,id_9998,Milan,['Italian'],4010.0,NaN,2.0,"[[], []]",/Restaurant_Review-g187849-d3862815-Reviews-La...,d3862815


In [18]:
data_test = pd.read_csv('kaggle_task.csv')
cui = pd.read_csv('kaggle_task.csv')
df = pd.read_csv('kaggle_task.csv')
cui['Cuisine Style'].fillna('Other', inplace = True)
cui['Cuisine Style'] = cui['Cuisine Style'].str.strip('[')
cui['Cuisine Style'] = cui['Cuisine Style'].str.strip(']')
cui['Cuisine Style'] = cui['Cuisine Style'].str.replace('\'','')
cui['Cuisine Style'] = cui['Cuisine Style'].str.split(',')
data_test['Cuisine Style'] = cui['Cuisine Style']
data_test = data_test.explode('Cuisine Style')
data_test = pd.get_dummies(data_test, columns=['Cuisine Style'], prefix='', prefix_sep='')
data_test = pd.get_dummies(data_test, columns=['City'], prefix='', prefix_sep='')
pattern = re.compile("\d+\/\d+\/\d+")
df['Reviews'] = df['Reviews'].fillna('None')
df['date_rev'] = df['Reviews'].apply(pattern.findall)
df['date_rev'] = df['date_rev'].apply(
lambda x: [x[-2], x[-1]] if len(x) >= 3 else x)
df['rev_1'] = df['date_rev'].apply(lambda x: x[0] if len(x) >= 1 else None)
df['rev_2'] = df['date_rev'].apply(lambda x: x[1] if len(x) == 2 else None)
df['rev_1'] = pd.to_datetime(df['rev_1'])
df['rev_2'] = pd.to_datetime(df['rev_2'])
def price(x):
    if x == '$$ - $$$':
        x = 2
    elif x =='$':
        x = 1
    else:
        x = 3
    return x
data_test['Price Range'] = data_test['Price Range'].apply(price)
data_test['days'] = abs(df['rev_1'] - df['rev_2']).dt.days
data_test['days'].fillna(data_test['days'].mean(),inplace = True)
df['rev_1'].fillna(df['rev_1'].mean(),inplace = True)
df['rev_1'] = df['rev_1'].apply(lambda x: mktime(x.timetuple()))
df['rev_2'].fillna(df['rev_2'].mean(),inplace = True)
df['rev_2'] = df['rev_2'].apply(lambda x: mktime(x.timetuple()))
data_test['rev_1'] = df['rev_1']
data_test['rev_2'] = df['rev_2']
use_later = data_test['Restaurant_id']
data_test.drop(['Restaurant_id','Reviews', 'URL_TA', 'ID_TA'],axis = 1,inplace = True)
data_test['Ranking'].fillna(data_test['Ranking'].mean(),inplace = True)
data_test['Number of Reviews'].fillna(data_test['Number of Reviews'].mean(),inplace = True)

In [19]:
skipped = []
num = []
col_1 = data_file.columns
col_2 = data_test.columns
ar = col_1.isin(col_2)
for i in range(len(ar)):
    if ar[i] == False:
        num.append(i)
for i in num:
    skipped.append(col_1[i])
skipped

[' Albanian',
 ' Burmese',
 ' Caucasian',
 ' Central Asian',
 ' Fujian',
 ' Latvian',
 ' New Zealand',
 ' Polynesian',
 ' Salvadoran',
 ' Uzbek',
 ' Xinjiang',
 'Balti',
 'Central American',
 'Central Asian',
 'Colombian',
 'Croatian',
 'Cuban',
 'Ethiopian',
 'Georgian',
 'Halal',
 'Hawaiian',
 'Indonesian',
 'Kosher',
 'Malaysian',
 'Mongolian',
 'Romanian',
 'Southwestern',
 'Ukrainian',
 'Uzbek']

In [20]:
for i in skipped:
    data_test[i] = 0
data_test

,Ranking,Price Range,Number of Reviews,Afghani,African,American,Arabic,Argentinean,Armenian,Asian,...,Halal,Hawaiian,Indonesian,Kosher,Malaysian,Mongolian,Romanian,Southwestern,Ukrainian,Uzbek
0,12963.0,2,4.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,12963.0,2,4.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,106.0,2,97.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,106.0,2,97.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,106.0,2,97.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,2612.0,2,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,2612.0,2,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,5562.0,3,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,4010.0,3,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
predict_submission = model.predict(data_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [31]:
values = []
predict_submission = predict_submission.tolist()

In [32]:
submission = pd.DataFrame()
submission['Restaurant_id'] = use_later
submission['Rating'] = predict_submission

In [33]:
submission = submission.groupby('Restaurant_id', as_index=False).agg(lambda x: x.tolist())

In [42]:
submission['Rating'] = submission['Rating'].apply(lambda x: x[0])

In [43]:
submission

,Restaurant_id,Rating
0,id_0,3.835
1,id_1,4.530
2,id_10,4.030
3,id_100,4.860
4,id_1000,4.320
...,...,...
9995,id_9995,4.405
9996,id_9996,4.995
9997,id_9997,4.050
9998,id_9998,4.965


In [28]:
submission.to_csv('submission.csv', index=False)